In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds

(train_ds, val_ds), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_ds = train_ds.map(preprocess).batch(BATCH_SIZE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE)

NUM_CLASSES = ds_info.features['label'].num_classes
print("Number of Classes:", NUM_CLASSES)


Number of Classes: 5


In [ ]:
# Custom AlexNet implementation
model_alexnet = models.Sequential([
    layers.Conv2D(96, (11,11), strides=4, activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((3,3), strides=2),
    layers.Conv2D(256, (5,5), padding='same', activation='relu'),
    layers.MaxPooling2D((3,3), strides=2),
    layers.Conv2D(384, (3,3), padding='same', activation='relu'),
    layers.Conv2D(384, (3,3), padding='same', activation='relu'),
    layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((3,3), strides=2),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model_alexnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_alexnet = model_alexnet.fit(train_ds, validation_data=val_ds, epochs=5)

alexnet_acc = model_alexnet.evaluate(val_ds)[1] * 100
print(f"✅ AlexNet Accuracy: {alexnet_acc:.2f}%")
